# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('previsao_de_renda.csv').drop(columns = ['Unnamed: 0','data_ref','id_cliente'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


## 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [4]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

## 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [5]:
modelo = '''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + idade
                + tempo_emprego '''
md = smf.ols(modelo, data = df_test)

alphas = [0,0.001,0.005,0.01,0.05,0.1]

for i in alphas:
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 0.0001
                             , alpha = i)
    print(reg.rsquared)


0.343994076035383
0.343994076035383
0.343994076035383
0.343994076035383
0.343994076035383
0.343994076035383


#### <span style="color: blue;"> Resposta: </span> Parece que todos os modelos tem o mesmo R², sendo assim nenhum é melhor.

## 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [6]:
modelo = '''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + idade
                + tempo_emprego '''
md = smf.ols(modelo, data = df_test)

alphas = [0,0.001,0.005,0.01,0.05,0.1]

for i in alphas:
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 1
                             , alpha = i)
    print(reg.rsquared)

0.343994076035383
0.343994076035383
0.3426013570257953
0.3426013570257953
0.3402929648471521
0.3402929648471521


#### <span style="color: blue;"> Resposta: </span> Os alphas de valor menor (0 e 0.001) chegam a R² maiores (que são iguais aos R² via regularização de ridge).

## 4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [7]:
df_test.posse_de_veiculo = [int(x) for x in df_test.posse_de_veiculo]
df_test.posse_de_imovel = [int(x) for x in df_test.posse_de_imovel]

In [8]:
X = df_test.drop('renda',axis=1)
y = df_test.renda
X

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia
10155,F,0,1,1,Empresário,Secundário,União,Casa,40,0.945205,3.0
11689,F,1,1,1,Assalariado,Secundário,Casado,Casa,34,1.347945,3.0
12599,F,1,1,1,Empresário,Superior completo,União,Casa,30,0.367123,3.0
12193,F,0,0,0,Assalariado,Secundário,União,Casa,27,4.767123,2.0
8678,M,0,1,1,Assalariado,Secundário,Casado,Casa,51,4.035616,3.0
...,...,...,...,...,...,...,...,...,...,...,...
10711,F,0,1,0,Pensionista,Secundário,Casado,Casa,65,NaN,2.0
12159,F,0,1,1,Empresário,Superior incompleto,Casado,Casa,28,7.167123,3.0
12814,M,0,0,0,Empresário,Superior completo,Casado,Casa,42,3.098630,2.0
10459,F,0,1,0,Pensionista,Secundário,Casado,Casa,62,NaN,2.0


In [9]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

## 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

## 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

## 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.